Beam-search

- https://habr.com/ru/articles/599673/
- https://habr.com/ru/articles/745314/
- https://habr.com/ru/articles/346578/
- https://habr.com/ru/companies/vk/articles/579412/
- https://habr.com/ru/companies/sberdevices/articles/666420/

Beam Search — это эвристический алгоритм поиска, широко применяемый в задачах обработки естественного языка (Natural Language Processing, NLP). Основная цель этого метода заключается в поиске оптимальной последовательности токенов (слов или символов) в условиях огромного количества возможных вариантов. Давайте рассмотрим подробнее, как Beam Search применяется в контексте NLP.
Что такое Beam Search?

Beam Search — это способ поиска, основанный на эвристическом подходе, который пытается найти лучшую последовательность элементов (например, слов в предложении) путём рассмотрения ограниченного числа возможных путей (или "лучей") на каждом этапе. Идея состоит в том, чтобы сосредоточиться лишь на небольшом количестве наиболее перспективных вариантов, игнорируя остальные. Этот метод существенно сокращает пространство поиска, делая возможным эффективное нахождение хороших решений в ситуациях, когда полное переборное исследование всех возможных вариантов невозможно.
Как работает Beam Search в NLP?

Представим себе задачу автоматического формирования текста или машинного перевода. Задача состоит в том, чтобы на основе некоторого начального ввода (например, "начало предложения") найти наилучшую последовательность слов, образующих осмысленное предложение. Рассмотрим пошагово, как это происходит с применением Beam Search:

- Инициализация: Начинаем с начального состояния (например, пустого ввода или первого слова).
- Генерация гипотез: На каждом шаге мы генерируем возможные продолжения для текущего состояния. В случае языковой модели это могут быть следующие слова, которые могут быть использованы в следующем токене.
- Оценка гипотез: Каждое возможное продолжение оценивается по некоторой метрике (например, по вероятности того, что именно это слово должно идти дальше согласно модели).
- Выбор лучших гипотез: Из всех возможных продолжений выбирается ограниченное количество лучших (обычно это параметр beam_width, задающий ширину пучка). Остальные гипотезы отбрасываются.
- Рекурсия: Процесс повторяется для каждого выбранного продолжения до тех пор, пока не будет достигнут конец предложения или другая заранее определённая точка остановки.
- Окончание: Когда поиск завершён, возвращаются лучшие последовательности, найденные методом Beam Search.

Плюсы и минусы Beam Search в NLP
Плюсы:

- Эффективность: Позволяет находить хорошие решения в больших пространствах состояний без полного перебора.
- Контролируемая сложность: Параметр beam_width даёт возможность настраивать компромисс между точностью и производительностью.
- Подходит для сложных задач: Используется в широком спектре приложений, включая машинный перевод, автоматическое формулирование текста и др.

Минусы:

- Не гарантирует оптимальное решение: Может пропустить лучшие варианты, если они находятся вне текущего пучка.
- Чувствителен к параметрам: Неправильно подобранная ширина пучка может привести либо к пропуску хороших решений, либо к чрезмерному увеличению времени вычисления.
- Требует настройки: Подбор правильных параметров (ширина пучка, длина последовательности и т.п.) требует экспериментов и тестов.

In [1]:
from nltk.corpus import gutenberg
from nltk.tokenize import word_tokenize
import torch.nn as nn
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import string
import torch.nn.functional as F
from collections import namedtuple

## Задача
1. Собрать полный пайплан по генерации текста с примением процедуры препроцессинга, токенизации и генерации используя алгоритм beam search. Попытаться добитьсякачественной генерации текста. За основу взять лабораторную №3. Попытаться реализовать как можно болеьше предложенных методов.   

In [2]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [3]:
file_ids = gutenberg.fileids()
texts_tokens = {} 

for fileid in file_ids:
    text = gutenberg.raw(fileid)
    tokens = [token.lower() for token in word_tokenize(text) if token not in string.punctuation]
    texts_tokens[fileid] = tokens

all_tokens = [token for tokens in texts_tokens.values() for token in tokens]
vocab = {token: idx for idx, token in enumerate(sorted(set(all_tokens)))}

print("Размер словаря:", len(vocab))

Размер словаря: 52494


In [4]:
texts_indices = {}
for fileid, tokens in texts_tokens.items():
    indices = [vocab[token] for token in tokens]
    texts_indices[fileid] = indices

print(texts_indices[file_ids[0]][:20])

[18621, 11210, 27723, 8239, 1914, 50340, 25779, 12151, 25779, 18621, 51871, 23995, 12903, 7047, 39668, 51719, 5517, 13357, 25209, 7047]


In [5]:
sample_file = list(texts_indices.keys())[0]
data = texts_indices[sample_file]
seq_length = 30

class TextDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, idx):
        x = self.data[idx:idx+self.seq_length]
        y = self.data[idx+1: idx+self.seq_length+1]
        return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)

dataset = TextDataset(data, seq_length)
batch_size = 64
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [6]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

    def forward(self, x, hidden):
        embed = self.embedding(x)
        output, hidden = self.lstm(embed, hidden)
        output = self.fc(output)
        return output, hidden

    def init_hidden(self, batch_size):
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return (h0, c0)

vocab_size = len(vocab)
embedding_dim = 128
hidden_size = 256
num_layers = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_lstm = LSTMModel(vocab_size, embedding_dim, hidden_size, num_layers).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model_lstm.parameters(), lr=0.001)
num_epochs = 20

model_lstm.train()
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(dataloader):
        optimizer.zero_grad()
        inputs = inputs.to(device)
        targets = targets.to(device)
        current_batch_size = inputs.size(0)
        hidden = model_lstm.init_hidden(current_batch_size)
        hidden = (hidden[0].to(device), hidden[1].to(device))
        outputs, hidden = model_lstm(inputs, hidden)
        outputs = outputs.reshape(-1, vocab_size)
        targets = targets.reshape(-1)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        if batch_idx % 100 == 0:
            print(f"LSTM Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}, Loss: {loss.item():.4f}")
    avg_loss = epoch_loss / len(dataloader)
    print(f"LSTM Epoch {epoch+1} average loss: {avg_loss:.4f}")

LSTM Epoch 1/20, Batch 0, Loss: 10.8706
LSTM Epoch 1/20, Batch 100, Loss: 6.3852
LSTM Epoch 1/20, Batch 200, Loss: 6.2832
LSTM Epoch 1/20, Batch 300, Loss: 6.3258
LSTM Epoch 1/20, Batch 400, Loss: 6.2900
LSTM Epoch 1/20, Batch 500, Loss: 6.1726
LSTM Epoch 1/20, Batch 600, Loss: 6.0906
LSTM Epoch 1/20, Batch 700, Loss: 6.0129
LSTM Epoch 1/20, Batch 800, Loss: 6.0186
LSTM Epoch 1/20, Batch 900, Loss: 5.9063
LSTM Epoch 1/20, Batch 1000, Loss: 5.8071
LSTM Epoch 1/20, Batch 1100, Loss: 5.7049
LSTM Epoch 1/20, Batch 1200, Loss: 5.6198
LSTM Epoch 1/20, Batch 1300, Loss: 5.8166
LSTM Epoch 1/20, Batch 1400, Loss: 5.5659
LSTM Epoch 1/20, Batch 1500, Loss: 5.4893
LSTM Epoch 1/20, Batch 1600, Loss: 5.4529
LSTM Epoch 1/20, Batch 1700, Loss: 5.3597
LSTM Epoch 1/20, Batch 1800, Loss: 5.4809
LSTM Epoch 1/20, Batch 1900, Loss: 5.3500
LSTM Epoch 1/20, Batch 2000, Loss: 5.3125
LSTM Epoch 1/20, Batch 2100, Loss: 5.3245
LSTM Epoch 1/20, Batch 2200, Loss: 5.3821
LSTM Epoch 1/20, Batch 2300, Loss: 5.1215
LST

In [7]:
def generate_text_lstm(model, start_text, vocab, inv_vocab, device, gen_length=30):
    model.eval()
    tokens = word_tokenize(start_text)
    input_indices = [vocab.get(token, 0) for token in tokens]
    input_tensor = torch.tensor(input_indices, dtype=torch.long).unsqueeze(0).to(device)
    hidden = model.init_hidden(input_tensor.size(0))
    hidden = (hidden[0].to(device), hidden[1].to(device))
    generated_tokens = tokens.copy()
    for _ in range(gen_length):
        outputs, hidden = model(input_tensor, hidden)
        logits = outputs[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        next_token_idx = torch.multinomial(probs, num_samples=1).item()
        generated_tokens.append(inv_vocab[next_token_idx])
        input_tensor = torch.tensor([[next_token_idx]], dtype=torch.long).to(device)
    return " ".join(generated_tokens)

inv_vocab = {idx: token for token, idx in vocab.items()}
generated_lstm = generate_text_lstm(model_lstm, "Alice was beginning to get very tired", vocab, inv_vocab, device, gen_length=30)
print("Сгенерированный текст LSTM:\n", generated_lstm)

Сгенерированный текст LSTM:
 Alice was beginning to get very tired of eating and drinking and playing whist with his neighbours five times a week than upon family affection or any thing that home affords '' emma could not like what


In [8]:
BeamEntry = namedtuple('BeamEntry', ['sequence', 'hidden', 'sum_logprob'])

def generate_text_beam(
    model,
    start_text,
    vocab,
    inv_vocab,
    device,
    max_len=30,
    beam_width=5,
    alpha=0.7,
    repetition_penalty=1.1,
    ngram_block=3,
    temperature=1.0
):
    model.eval()
    init_tokens = [vocab.get(t, 0) for t in word_tokenize(start_text.lower())]
    hidden = model.init_hidden(1)
    hidden = (hidden[0].to(device), hidden[1].to(device))
    beam = [BeamEntry(init_tokens, hidden, 0.0)]
    for _ in range(max_len):
        cand = []
        for seq, h, s in beam:
            last = torch.tensor([[seq[-1]]], dtype=torch.long).to(device)
            with torch.no_grad():
                logits, h_new = model(last, h)
            logits = logits[:, -1, :].squeeze(0)
            for t in seq:
                logits[t] /= repetition_penalty
            if temperature != 1.0:
                logits = logits / temperature
            log_probs = F.log_softmax(logits, dim=-1)
            topk_logp, topk_idx = torch.topk(log_probs, beam_width)
            prev_ngrams = {
                tuple(seq[i : i + ngram_block])
                for i in range(len(seq) - ngram_block + 1)
            }
            for lp, idx in zip(topk_logp.tolist(), topk_idx.tolist()):
                nseq = seq + [idx]
                if tuple(nseq[-ngram_block:]) in prev_ngrams:
                    continue
                cand.append(
                    BeamEntry(nseq, (h_new[0].clone(), h_new[1].clone()), s + lp)
                )
        if not cand:
            break
        cand.sort(
            key=lambda e: e.sum_logprob
            / ((5 + len(e.sequence)) ** alpha / 5**alpha),
            reverse=True,
        )
        beam = cand[: beam_width]
    best = max(
        beam,
        key=lambda e: e.sum_logprob
        / ((5 + len(e.sequence)) ** alpha / 5**alpha),
    )
    return " ".join(inv_vocab.get(i, "<unk>") for i in best.sequence)

generated_beam = generate_text_beam(
    model_lstm,
    "Alice was beginning to get very tired",
    vocab,
    {v: k for k, v in vocab.items()},
    device,
    max_len=30,
    beam_width=5,
)
print(generated_beam)

alice was beginning to get very tired you will be so kind as to give her your arm. -- mr elton and miss hawkins -- good morning to you '' emma alone with her father had half


Дополнительная задача, на дополнительный бал:

Попытаться реализовать

Exhaustive Search

Exhaustive Search проверяет все возможные варианты, гарантируя нахождение абсолютно лучшего решения. Однако, это непрактично для большинства реальных задач из-за огромной вычислительной сложности.



In [ ]:
def dynamic_temperature(timestep, total_timesteps, initial_temp, final_temp):
    """
    Динамическая настройка температуры.

    Аргументы:
    timestep: Текущий временной шаг.
    total_timesteps: Общее количество временных шагов.
    initial_temp: Начальная температура.
    final_temp: Конечная температура.

    Возвращает:
    Значение температуры на данном временном шаге.
    """
    temp = initial_temp + (final_temp - initial_temp) * (timestep / total_timesteps)
    return temp

In [ ]:
def evaluate_bleu(reference, candidate):
    """
    Оценивает качество перевода с помощью метрики BLEU.

    Аргументы:
    reference: Список ссылок.
    candidate: Генерируемый перевод.

    Возвращает:
    Оценка BLEU.
    """
    bleu_score = sentence_bleu(reference, candidate)
    return bleu_score

In [ ]:
import math

def weighted_greedy_sampling(logits, beams, weights=None):
    """
    Алгоритм взвешенной жадной выборки токенов.

    Аргументы:
    logits: Логиты токенов (вероятности).
    beams: Количество выбираемых токенов.
    weights: Веса для каждого токена (по умолчанию равны 1).

    Возвращает:
    Индексы выбранных токенов.
    """
    if weights is None:
        weights = torch.ones_like(logits)

    # Нормализуем веса
    normalized_weights = weights / weights.sum()

    # Применяем взвешивание к логитам
    weighted_logits = logits * normalized_weights

    # Выбираем лучшие токены
    return torch.topk(weighted_logits, beams).indices

In [ ]:
def beam_search_with_length_penalty(input_ids, node, bar, length, beams, sampling, temperature=0.1, alpha=0.6):
    if length == 0:
        return None

    outputs = model(input_ids)
    predictions = outputs.logits

    logits = predictions[0, -1, :]

    if sampling == 'weighted_greedy':
        top_token_ids = weighted_greedy_sampling(logits, beams)
    else:
        raise ValueError(f"Unknown sampling method {sampling}")

    for j, token_id in enumerate(top_token_ids):
        bar.update(1)

        # Рассчитываем вероятность токена
        token_score = get_log_prob(logits, token_id)
        cumulative_score = graph.nodes[node]['cumscore'] + token_score

        # Добавляем предсказанный токен к списку входных идентификаторов
        new_input_ids = torch.cat([input_ids, token_id.unsqueeze(0).unsqueeze(0)], dim=-1)

        # Учет длины последовательности при расчете общей оценки
        sequence_length = len(new_input_ids.squeeze())
        length_penalty = ((sequence_length + 5) / 6)**alpha
        penalized_cumscore = cumulative_score / length_penalty

        # Обновляем граф
        token = tokenizer.decode(token_id, skip_special_tokens=True)
        current_node = list(graph.successors(node))[j]
        graph.nodes[current_node]['tokenscore'] = np.exp(token_score) * 100
        graph.nodes[current_node]['cumscore'] = cumulative_score
        graph.nodes[current_node]['penalized_cumscore'] = penalized_cumscore
        graph.nodes[current_node]['sequencescore'] = 1/(len(new_input_ids.squeeze())) * cumulative_score
        graph.nodes[current_node]['token'] = token + f"_{length}_{j}"

        # Рекурсивный вызов
        beam_search_with_length_penalty(
            new_input_ids,
            current_node,
            bar,
            length-1,
            beams,
            sampling,
            temperature,
            alpha
        )

In [ ]:
def get_best_sequence(G):
    # Create a list of leaf nodes
    leaf_nodes = [node for node in G.nodes() if G.out_degree(node)==0]

    # Get the leaf node with the highest cumscore
    max_score_node = None
    max_score = float('-inf')
    for node in leaf_nodes:
        if G.nodes[node]['sequencescore'] > max_score:
            max_score = G.nodes[node]['sequencescore']
            max_score_node = node

    # Retrieve the sequence of nodes from this leaf node to the root node in a list
    path = nx.shortest_path(G, source=0, target=max_score_node)

    # Return the string of token attributes of this sequence
    sequence = "".join([G.nodes[node]['token'].split('_')[0] for node in path])

    return sequence, max_score

sequence, max_score = get_best_sequence(graph)
print(f"Generated text: {sequence}")

In [ ]:
class AttentionLayer(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.query_proj = nn.Linear(hidden_size, hidden_size)
        self.key_proj = nn.Linear(hidden_size, hidden_size)
        self.value_proj = nn.Linear(hidden_size, hidden_size)

    def forward(self, query, keys, values):
        Q = self.query_proj(query)
        K = self.key_proj(keys)
        V = self.value_proj(values)
        attention_scores = torch.matmul(Q, K.transpose(-1, -2)) / math.sqrt(K.shape[-1])
        attention_probs = F.softmax(attention_scores, dim=-1)
        context_vector = torch.matmul(attention_probs, V)
        return context_vector

In [ ]:
def add_regularization(model, regularization_type='dropout'):
    """
    Добавляет регуляризацию к модели.

    Аргументы:
    model: Модель, к которой добавляется регуляризация.
    regularization_type: Тип регуляризации ('dropout' или другой).

    Возвращает:
    Регуляризированную модель.
    """
    if regularization_type == 'dropout':
        model.add_module('dropout', nn.Dropout(p=0.1))
    return model

In [ ]:
def add_regularization(model, regularization_type='dropout'):
    """
    Добавляет регуляризацию к модели.

    Аргументы:
    model: Модель, к которой добавляется регуляризация.
    regularization_type: Тип регуляризации ('dropout' или другой).

    Возвращает:
    Регуляризированную модель.
    """
    if regularization_type == 'dropout':
        model.add_module('dropout', nn.Dropout(p=0.1))
    return model

In [ ]:
def rank_sequences(graph, nodes):
    """
    Ранжирование последовательностей по различным критериям.

    Аргументы:
    graph: Граф, содержащий узлы с информацией о токенах.
    nodes: Список узлов, для которых производится ранжирование.

    Возвращает:
    Отсортированный список узлов по убыванию их общей оценки.
    """
    scores = []
    for node in nodes:
        cumscore = graph.nodes[node]['cumscore']
        sequencescore = graph.nodes[node]['sequencescore']
        diversity_score = calculate_diversity_score(graph, node)
        final_score = cumscore * sequencescore * diversity_score
        scores.append((node, final_score))

    sorted_nodes = sorted(scores, key=lambda x: x[1], reverse=True)
    return [node for node, _ in sorted_nodes]

def calculate_diversity_score(graph, node):
    """
    Расчет показателя разнообразия последовательности.

    Аргументы:
    graph: Граф, содержащий узлы с информацией о токенах.
    node: Узел, для которого рассчитывается показатель разнообразия.

    Возвращает:
    Показатель разнообразия последовательности.
    """
    tokens = []
    while node != 0:
        tokens.append(graph.nodes[node]['token'])
        node = list(graph.predecessors(node))[0]
    unique_tokens = set(tokens)
    diversity_score = len(unique_tokens) / len(tokens)
    return diversity_score

In [ ]:
def diverse_top_k_sampling(logits, k, diversity_rate=0.7):
    """
    Выборка токенов с учетом диверсификации.

    Аргументы:
    logits: Логиты токенов.
    k: Количество выбираемых токенов.
    diversity_rate: Коэффициент диверсификации (чем ближе к 1, тем сильнее влияние).

    Возвращает:
    Индексы отобранных токенов.
    """
    top_values, top_indices = torch.topk(logits, k)
    diversity_scores = compute_diversity_scores(top_indices)
    combined_scores = top_values * (diversity_rate * diversity_scores + (1 - diversity_rate))
    _, selected_indices = torch.topk(combined_scores, k)
    return top_indices[selected_indices]

In [ ]:
def train_on_the_fly(model, input_ids, labels, optimizer, loss_fn):
    """
    Обучение модели на лету.

    Аргументы:
    model: Модель, которую нужно обновить.
    input_ids: Входные данные.
    labels: Метки для входных данных.
    optimizer: Оптимизатор для обучения.
    loss_fn: Функция потерь.

    Возвращает:
    Потери после одного шага обучения.
    """
    outputs = model(input_ids)
    predictions = outputs.logits
    loss = loss_fn(predictions.view(-1, predictions.size(-1)), labels.view(-1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()

In [ ]:
def adaptive_temperature_adjustment(current_step, max_steps, initial_temp, min_temp):
    """
    Адаптивное управление температурой.

    Аргументы:
    current_step: Текущий шаг процесса.
    max_steps: Максимальное количество шагов.
    initial_temp: Начальная температура.
    min_temp: Минимальная температура.

    Возвращает:
    Температуру на текущем шаге.
    """
    progress = current_step / max_steps
    temp = initial_temp * (1 - progress) + min_temp * progress
    return temp

In [ ]:
import networkx as nx

# Создаем граф с весовыми коэффициентами на рёбрах и дополнительными параметрами для узлов
G = nx.DiGraph()

# Пример добавления узлов с дополнительным параметром 'context'
G.add_node('A', sequencescore=10, context='Начало предложения')
G.add_node('B', sequencescore=20, context='Основная часть предложения')
G.add_node('C', sequencescore=30, context='Конец предложения')

# Добавляем рёбра с весами
G.add_edge('A', 'B', weight=2)
G.add_edge('B', 'C', weight=4)
G.add_edge('A', 'C', weight=8)

# Функция для поиска оптимального пути через граф с учетом весов
def get_best_sequence_with_weights(G):
    # Получаем листовые узлы (листья)
    leaf_nodes = [node for node in G.nodes() if G.out_degree(node) == 0]

    # Находим листовой узел с максимальным score
    best_leaf_node = None
    best_score = float('-inf')
    for node in leaf_nodes:
        if G.nodes[node]['sequencescore'] > best_score:
            best_score = G.nodes[node]['sequencescore']
            best_leaf_node = node

    # Используем алгоритм Дейкстры для нахождения кратчайшего пути до листового узла
    try:
        shortest_path = nx.dijkstra_path(G, source='A', target=best_leaf_node, weight='weight')
    except nx.NetworkXNoPath:
        print("Нет пути от начального узла до листового!")
        return None, None

    # Формируем последовательность текста
    sequence = ''.join([G.nodes[node]['token'].split('_')[0] for node in shortest_path])

    return sequence, best_score

# Вызываем функцию
sequence, max_score = get_best_sequence_with_weights(G)
if sequence is not None:
    print(f"Оптимальная последовательность: {sequence}, максимальная оценка: {max_score}")